In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Optionally, check if the key is loaded
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("GOOGLE_API_KEY not found in .env file.")

In [2]:
os.listdir("../../docs")

['requirements.md']

In [3]:
from crewai import LLM
from crewai_tools import WebsiteSearchTool
# Read your API key from the environment variable
gemini_api_key = os.getenv("GOOGLE_API_KEY")

# Use Gemini 2.5 Pro Experimental model
gemini_llm = LLM(
    model='gemini/gemini-2.5-pro',
    api_key=gemini_api_key,
    temperature=0.0  # Lower temperature for more consistent results.
)



/Users/yunweihu/miniconda/lib/python3.11/site-packages/pydantic/fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
/Users/yunweihu/miniconda/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [4]:
from crewai.tools import BaseTool

from crewai_tools import FileReadTool
read_requirements_tool = FileReadTool( "../../docs/requirements.md")  # File containing requirements

# Placeholder tool for fetching functional requirements from ./requirements directory. Using crewai_tools for simplicity.
class ReadRequirementsTool(BaseTool):
    name: str = "Functional Requirements Reader"
    description: str = (
        "Reads functional requirements from the ./requirements directory. "
        "Returns a summary string of the requirements.")

    def _run(self, argument: str) -> str:
        # In a real scenario, this would read from files in the ./requirements directory.
        # For this example, return simulated data.
        print(f"--- Reading requirements for query: {argument} ---")
        return (
            """Functional Requirements Summary:""")

# Placeholder tool for fetching customer support data
class CustomerSupportDataTool(BaseTool):
    name: str = "Customer Support Data Fetcher"
    description: str = (
      "Fetches recent customer support interactions, tickets, and feedback. "
      "Returns a summary string.")

    def _run(self, argument: str) -> str:
        # In a real scenario, this would query a database or API.
        # For this example, return simulated data.
        print(f"--- Fetching data for query: {argument} ---")
        return (
            """Recent Support Data Summary:
- 50 tickets related to 'login issues'. High resolution time (avg 48h).
- 30 tickets about 'billing discrepancies'. Mostly resolved within 12h.
- 20 tickets on 'feature requests'. Often closed without resolution.
- Frequent feedback mentions 'confusing user interface' for password reset.
- High volume of calls related to 'account verification process'.
- Sentiment analysis shows growing frustration with 'login issues' resolution time.
- Support agent notes indicate difficulty reproducing 'login issues'."""
        )

support_data_tool = CustomerSupportDataTool()

Agents


In [5]:
from crewai import Agent

# Agent 1: function analyst
function_analyst = Agent(
    role='Functional Analyst',
    goal='Analyze Functional Requirements of the system of interest. Get ready for the next step of the process, which is to generate failuare modes for each function.',
    backstory=(
        """You are a Senior Systems Reliability Engineer with deep expertise in Failure Mode and Effects Analysis (FMEA) for {product}. Your primary role is to ensure that functional requirements are written with enough clarity, precision, and verifiability to serve as a solid foundation for identifying potential failure modes. You are an expert in applying the principles from the INCOSE Guide for Writing Requirements."""
    ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[read_requirements_tool],  # Assign the data fetching tool
    llm=gemini_llm  # Use the configured Gemini LLM
)

failure_modes_analyst = Agent(
    role='Reliability and FMEA Analyst',
    goal='Generate a comprehensive and structured list of potential failure modes derived from a given set of technical functional requirements.',
    backstory=(
    "You are a Senior Reliability Engineer and a seasoned FMEA (Failure Mode and Effects Analysis) facilitator. "
    "With a deep background in {product} and systems engineering, you are an expert at dissecting functional requirements to anticipate every conceivable way a component or system could fail. Your work is meticulous, systematic, and forms the critical foundation for robust and safe engineering design. You live and breathe standards for risk analysis and reliability."
  ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[],  # Assign the data fetching tool
    llm=gemini_llm  # Use the configured Gemini LLM
)

Tasks


In [6]:
from crewai import Task

# Task 1: Analyze functional requirements
functional_analysis_task = Task(
    description=(
        """Fetch and analyze the functional requirements. Use the `read_requirements_tool`. Review each functional requirement individually and assess its readiness for an FMEA. For each requirement, perform the following analysis based on the core principles outlined in the INCOSE Guide for Writing Requirements:  
1. Assess Verifiability (C7): Is the requirement stated in a way that is measurable and testable? Can a clear pass/fail criterion be established? If not, explain what is missing.  
2. Assess Singularity/Atomicity (C5): Does the requirement address only one single function or characteristic? Identify any "and/or" conjunctions or phrases that combine multiple requirements into one statement. For the FMEA to be effective, we need to analyze the failure of one function at a time.  
3. Assess Unambiguity (C3): Is the language clear, precise, and free of vague terms? Identify any words like "approximately," "user-friendly," "robust," "flexible," or "without degradation" that could be interpreted in multiple ways. A failure mode must be derived from a precise function.  
4. Failure Mode Derivation: For each requirement, state whether a clear failure mode can be derived.  
- If YES: Provide an example of a primary failure mode (e.g., "Fails to maintain capacitance within tolerance," "Fails to withstand rated voltage").  
- If NO: Explain why a clear failure mode cannot be derived and suggest a specific improvement to the requirement's wording.  
5. Provide Recommendations: For any requirement that does not meet the criteria, provide a rewritten version that is atomic, verifiable, and unambiguous, making it FMEA-ready.  
6. Implement the recommendations: Update the list of functional requirements. Do not add or delete any requirements, just modify the existing ones to meet the criteria.
"""
    ),
    expected_output=(
        """Update the list of functional requirements. Do not add or delete any requireemnts. Output a markdown table with the following columns:
- Req. ID: The unique identifier for the functional requirement.
- Original Requirement: The original functional requirement text.
- Revised Requirement: The revised functional requirement text that meets the criteria.
- Verifiability: Assessment of whether the requirement is measurable and testable.
- Singularity/Atomicity: Assessment of whether the requirement is atomic and addresses a single function.
- Unambiguity: Assessment of whether the requirement is clear and free of vague terms.
- Failure Mode Derivation: Whether a clear failure mode can be derived from the requirement.
- Example Failure Mode: If applicable, provide an example of a primary failure mode derived from the requirement.
- Recommendations: Any suggestions for improving the requirement's wording to make it FMEA-ready."""
    ),
    agent=function_analyst  # Assign task to the data_analyst agent
)

failure_modes_generation = Task(
  description=(
    "You are conducting a component-level FMEA on a capacitor for a VFD system. "
    "Your task is to generate a comprehensive list of potential failure modes for each function listed in the requirements. "
    "A 'failure mode' is the specific manner in which a component can fail to meet its intended function.\n\n"
    "**FRAMEWORK & GUIDE WORDS:**\n"
    "To ensure a thorough analysis, you MUST systematically apply the following generic failure mode categories as 'guide words' "
    "to each functional requirement. For each requirement, consider how it could fail in each of these ways:\n"
    "1.  **Loss of Function** (Total failure)\n"
    "2.  **Degraded Function** (Partial failure: Over Specification [Too much/high] or Under Specification [Too little/low])\n"
    "3.  **Intermittent Function**\n"
    "4.  **Unintended Function**\n"
    "5.  **Erroneous Timing** (Too late / Too early)\n"
    "6.  **Erroneous Duration** (Too long / Too short)\n"
    "7.  **Conditional Failure** (Fails only under specific, allowable operating conditions)\n\n"

    "**INSTRUCTIONS:**\n"
    "**Use the updated functional requirements provided in the context from the previous step**.  "
    "For each functional requirement, generate a list of specific, plausible failure modes. "
    "The failure modes you generate must be directly and technically related to the function described in the requirement. "
    "Be precise. For example, for FR-1, instead of just 'Degraded Function,' a specific failure mode would be "
    "'Capacitance is below the specified lower tolerance limit.' Apply all relevant guide words to each requirement."
  ),
  expected_output=(
    "A complete and detailed markdown table with four columns: 'Req. ID', 'Functional Requirement', 'Generic Failure Mode Category', and 'Specific Failure Mode(s)'. "
    "The table must be exhaustive, listing all plausible failure modes for EVERY provided requirement, categorized according to the given framework. "
    "The 'Specific Failure Mode(s)' column must contain clear, technical descriptions of the failure."
  ),
  agent=failure_modes_analyst,
  context=[functional_analysis_task],
  )



Crew

In [7]:
from crewai import Crew, Process

# Define the crew with agents, tasks, and process
fmea_crew = Crew(
    agents=[function_analyst, failure_modes_analyst],
    tasks=[functional_analysis_task, failure_modes_generation],
    process=Process.sequential,  # Tasks will run sequentially in the order defined
    verbose=True
)

Run

In [8]:
# Start the crew's work
print("--- Starting FMEA Advisor Crew ---")
# The 'inputs' dictionary provides initial context if needed by the first task.
# In this case, the tool simulates data fetching regardless of the input.
result =fmea_crew.kickoff(inputs={'product': 'Variable Frequency Drive (VFD)'})

print("--- Crew Execution Finished ---")
print("--- Final Report ---")


--- Starting FMEA Advisor Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 01de2ed4-46d5-4bb0-adca-2db411adf630                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Functional Analyst                                                                                      │
│                                                                                                                 │
│  Task: Fetch and analyze the functional requirements. Use the `read_requirements_tool`. Review each functional  │
│  requirement individually and assess its readiness for an FMEA. For each requirement, perform the following     │
│  analysis based on the core principles outlined in the INCOSE Guide for Writing Requirements:                   │
│  1. Assess Verifiability (C7): Is the requirement stated in a way that is measurable and testable? Can a clear  │
│  pass/fail criterion be established? If not, explain what is missing.                                           │
│  2. Assess Singularity/Atomicity (C5): Does the requirement address only one single function or                 │
│  characteristic? Identify any "and/or" conjunctions or phrases that combine multiple requirements into one      │
│  statement. For the FMEA to be effective, we need to analyze the failure of one function at a time.             │
│  3. Assess Unambiguity (C3): Is the language clear, precise, and free of vague terms? Identify any words like   │
│  "approximately," "user-friendly," "robust," "flexible," or "without degradation" that could be interpreted in  │
│  multiple ways. A failure mode must be derived from a precise function.                                         │
│  4. Failure Mode Derivation: For each requirement, state whether a clear failure mode can be derived.           │
│  - If YES: Provide an example of a primary failure mode (e.g., "Fails to maintain capacitance within            │
│  tolerance," "Fails to withstand rated voltage").                                                               │
│  - If NO: Explain why a clear failure mode cannot be derived and suggest a specific improvement to the          │
│  requirement's wording.                                                                                         │
│  5. Provide Recommendations: For any requirement that does not meet the criteria, provide a rewritten version   │
│  that is atomic, verifiable, and unambiguous, making it FMEA-ready.                                             │
│  6. Implement the recommendations: Update the list of functional requirements. Do not add or delete any         │
│  requirements, just modify the existing ones to meet the criteria.                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Functional Analyst                                                                                      │
│                                                                                                                 │
│  Thought: Thought: I need to start by fetching the functional requirements. I will use the `Read a file's       │
│  content` tool to read the default file `../../docs/requirements.md`. After reading the file, I will analyze    │
│  each requirement according to the user's instructions.                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Reliability and FMEA Analyst                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Component FMEA: Potential Failure Modes for VFD Capacitor                                                  │
│                                                                                                                 │
│  The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived    │
│  systematically from the provided functional requirements using the specified guide words.                      │
│                                                                                                                 │
│  | Req. ID | Functional Requirement | Generic Failure Mode Category | Specific Failure Mode(s) |                │
│  | :--- | :--- | :--- | :--- |                                                                                  │
│  | **FR-1** | The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% at 60 Hz and  │
│  35°C. | **Loss of Function** | • Capacitor fails open (capacitance drops to near zero).<br>• Capacitor fails   │
│  short (dielectric breakdown results in a low-resistance path). |                                               │
│  | | | **Degraded Function (Under)** | • Capacitance is below the lower tolerance limit of 95 µF due to aging,  │
│  electrolyte evaporation, or internal damage. |                                                                 │
│  | | | **Degraded Function (Over)** | • Capacitance is above the upper tolerance limit of 105 µF due to         │
│  manufacturing defect or dielectric property shift. |                                                           │
│  | | | **Intermittent Function** | • Capacitance value intermittently falls outside the specified ±5%           │
│  tolerance range due to a poor internal connection or temperature instability. |                                │
│  | | | **Unintended Function** | • Capacitor exhibits significant inductive properties at operating             │
│  frequency.<br>• Capacitor acts primarily as a resistor due to catastrophic ESR increase. |                     │
│  | | | **Conditional Failure** | • Capacitance is within tolerance at 35°C but falls out of tolerance at other  │
│  temperatures within the allowable operating range (e.g., at 0°C or 60°C). |                                    │
│  | **FR-2** | The capacitor shall have a rated DC working voltage of no less than 5 VDC. | **Loss of            │
│  Function** | • Dielectric breakdown (short circuit) occurs at a voltage below the 5 VDC rating. |              │
│  | | | **Degraded Function (Under)** | • Insulation resistance is significantly lower than specified at the     │
│  rated voltage, leading to excessive leakage current (precursor to failure). |                                  │
│  | | | **Intermittent Function** | • Intermittent arcing or temporary shorting occurs when voltage approaches   │
│  the 5 VDC rating. |                                                                                            │
│  | | | **Erroneous Duration** | • Withstands 5 VDC for a short duration but fails (shorts) before the expected  │
│  lifetime under continuous voltage application. |                                                               │
│  | | | **Conditional Failure** | • Withstands 5 VDC at nominal temperature but fails (shorts) at the maximum    │
│  rated operating temperature (e.g., 60°C). |           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2e726af4-6e04-42b9-88b6-9c75f70f7469                                                                     │
│  Agent: Reliability and FMEA Analyst                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 01de2ed4-46d5-4bb0-adca-2db411adf630                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Component FMEA: Potential Failure Modes for VFD Capacitor                                    │
│                                                                                                                 │
│  The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived    │
│  systematically from the provided functional requirements using the specified guide words.                      │
│                                                                                                                 │
│  | Req. ID | Functional Requirement | Generic Failure Mode Category | Specific Failure Mode(s) |                │
│  | :--- | :--- | :--- | :--- |                                                                                  │
│  | **FR-1** | The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% at 60 Hz and  │
│  35°C. | **Loss of Function** | • Capacitor fails open (capacitance drops to near zero).<br>• Capacitor fails   │
│  short (dielectric breakdown results in a low-resistance path). |                                               │
│  | | | **Degraded Function (Under)** | • Capacitance is below the lower tolerance limit of 95 µF due to aging,  │
│  electrolyte evaporation, or internal damage. |                                                                 │
│  | | | **Degraded Function (Over)** | • Capacitance is above the upper tolerance limit of 105 µF due to         │
│  manufacturing defect or dielectric property shift. |                                                           │
│  | | | **Intermittent Function** | • Capacitance value intermittently falls outside the specified ±5%           │
│  tolerance range due to a poor internal connection or temperature instability. |                                │
│  | | | **Unintended Function** | • Capacitor exhibits significant inductive properties at operating             │
│  frequency.<br>• Capacitor acts primarily as a resistor due to catastrophic ESR increase. |                     │
│  | | | **Conditional Failure** | • Capacitance is within tolerance at 35°C but falls out of tolerance at other  │
│  temperatures within the allowable operating range (e.g., at 0°C or 60°C). |                                    │
│  | **FR-2** | The capacitor shall have a rated DC working voltage of no less than 5 VDC. | **Loss of            │
│  Function** | • Dielectric breakdown (short circuit) occurs at a voltage below the 5 VDC rating. |              │
│  | | | **Degraded Function (Under)** | • Insulation resistance is significantly lower than specified at the     │
│  rated voltage, leading to excessive leakage current (precursor to failure). |                                  │
│  | | | **Intermittent Function** | • Intermittent arcing or temporary shorting occurs when voltage approaches   │
│  the 5 VDC rating. |                                                                                            │
│  | | | **Erroneous Duration** | • Withstands 5 VDC for a short duration but fails (shorts) before the expected  │
│  lifetime under continuous voltage application. |                                                               │
│  | | | **Conditional Failure** | • Withstands 5 VDC at

--- Crew Execution Finished ---
--- Final Report ---


In [9]:
from IPython.display import display, Markdown

# Extract the raw text from the result object
if hasattr(result, "raw"):
    display(Markdown(result.raw))
else:
    raise TypeError("Result object does not contain raw text for Markdown.")

### Component FMEA: Potential Failure Modes for VFD Capacitor

The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived systematically from the provided functional requirements using the specified guide words.

| Req. ID | Functional Requirement | Generic Failure Mode Category | Specific Failure Mode(s) |
| :--- | :--- | :--- | :--- |
| **FR-1** | The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% at 60 Hz and 35°C. | **Loss of Function** | • Capacitor fails open (capacitance drops to near zero).<br>• Capacitor fails short (dielectric breakdown results in a low-resistance path). |
| | | **Degraded Function (Under)** | • Capacitance is below the lower tolerance limit of 95 µF due to aging, electrolyte evaporation, or internal damage. |
| | | **Degraded Function (Over)** | • Capacitance is above the upper tolerance limit of 105 µF due to manufacturing defect or dielectric property shift. |
| | | **Intermittent Function** | • Capacitance value intermittently falls outside the specified ±5% tolerance range due to a poor internal connection or temperature instability. |
| | | **Unintended Function** | • Capacitor exhibits significant inductive properties at operating frequency.<br>• Capacitor acts primarily as a resistor due to catastrophic ESR increase. |
| | | **Conditional Failure** | • Capacitance is within tolerance at 35°C but falls out of tolerance at other temperatures within the allowable operating range (e.g., at 0°C or 60°C). |
| **FR-2** | The capacitor shall have a rated DC working voltage of no less than 5 VDC. | **Loss of Function** | • Dielectric breakdown (short circuit) occurs at a voltage below the 5 VDC rating. |
| | | **Degraded Function (Under)** | • Insulation resistance is significantly lower than specified at the rated voltage, leading to excessive leakage current (precursor to failure). |
| | | **Intermittent Function** | • Intermittent arcing or temporary shorting occurs when voltage approaches the 5 VDC rating. |
| | | **Erroneous Duration** | • Withstands 5 VDC for a short duration but fails (shorts) before the expected lifetime under continuous voltage application. |
| | | **Conditional Failure** | • Withstands 5 VDC at nominal temperature but fails (shorts) at the maximum rated operating temperature (e.g., 60°C). |
| **FR-3** | The capacitor shall withstand a continuous root-mean-square (RMS) ripple current of 0.1 A at 60 Hz and a maximum ambient temperature of 40°C. | **Loss of Function** | • Catastrophic failure (venting, shorting, or opening) occurs when subjected to the 0.1 A RMS ripple current. |
| | | **Degraded Function (Under)** | • Withstands 0.1 A ripple current, but internal temperature rises excessively, leading to accelerated degradation of capacitance, ESR, and lifetime. |
| | | **Unintended Function** | • Generates excessive heat, acting as a resistive heating element.<br>• Generates audible noise (humming) under the specified ripple current load. |
| | | **Erroneous Duration** | • Fails to withstand the 0.1 A ripple current continuously; fails after a short period of operation. |
| | | **Conditional Failure** | • Withstands 0.1 A ripple current at room temperature but fails (overheats, vents) at the specified 40°C ambient condition. |
| **FR-4** | The capacitor shall have a maximum Equivalent Series Resistance (ESR) of 100 mΩ when measured at 60 Hz and 35°C. | **Loss of Function** | • ESR increases to a level where the capacitor effectively functions as an open circuit to AC signals. |
| | | **Degraded Function (Over)** | • ESR exceeds the maximum specified limit of 100 mΩ at the specified test conditions. |
| | | **Intermittent Function** | • ESR value fluctuates, intermittently exceeding the 100 mΩ limit due to a faulty internal weld or lead connection. |
| | | **Erroneous Duration** | • ESR is initially within specification but rises above 100 mΩ after a short operational period. |
| | | **Conditional Failure** | • ESR is within specification at 35°C but exceeds 100 mΩ at the lower or upper limits of the operating temperature range (0°C or 60°C). |
| **FR-5** | The capacitor shall maintain all specified electrical characteristics (Capacitance, ESR, Leakage Current) within their defined limits across an ambient operating temperature range of 0°C to 60°C. | **Loss of Function** | • Capacitor fails catastrophically (shorts or opens) when operated at or near the temperature limits of 0°C or 60°C. |
| | | **Degraded Function** | • Capacitance falls below 95 µF when operated at 0°C.<br>• ESR exceeds 100 mΩ when operated at 60°C.<br>• Leakage current exceeds 10 µA when operated at 60°C. |
| | | **Intermittent Function** | • Electrical parameters become unstable or fluctuate in and out of specification when operating at a sustained temperature within the range. |
| | | **Unintended Function** | • Physical deformation (case bulging, seal compromise, electrolyte leakage) occurs at the high end of the temperature range without immediate electrical failure. |
| **FR-6** | The capacitor shall have a maximum leakage current of 10 µA after 1 minute of operation at the rated DC working voltage and 35°C. | **Loss of Function** | • Leakage current is excessively high, creating a partial short circuit path. |
| | | **Degraded Function (Over)** | • Leakage current exceeds the 10 µA limit under the specified test conditions. |
| | | **Intermittent Function** | • Leakage current is typically below the limit but intermittently spikes above 10 µA. |
| | | **Erroneous Timing (Too late)** | • Leakage current takes significantly longer than 1 minute to stabilize below the 10 µA threshold. |
| | | **Erroneous Duration** | • Leakage current is initially below 10 µA but drifts above the limit during continuous operation. |
| | | **Conditional Failure** | • Leakage current is within specification at 35°C but exceeds 10 µA at higher allowable operating temperatures (e.g., 60°C). |
| **FR-7** | The capacitor's physical dimensions shall be contained within a cylindrical envelope of 10 mm diameter and 20 mm height. | **Degraded Function (Over)** | • Diameter exceeds 10 mm.<br>• Height exceeds 20 mm. |
| | | **Unintended Function** | • Body of the capacitor is not cylindrical (e.g., ovalized, dented), preventing proper automated assembly or fit. |
| | | **Conditional Failure** | • Dimensions are within specification at room temperature but expand beyond the envelope due to thermal expansion at the maximum operating temperature. |
| **FR-8** | The capacitor shall have a lead spacing of 5 mm ±0.5 mm. | **Degraded Function (Over)** | • Lead spacing is greater than 5.5 mm. |
| | | **Degraded Function (Under)** | • Lead spacing is less than 4.5 mm. |
| | | **Unintended Function** | • Leads are not parallel to each other.<br>• Leads are bent, damaged, or have insufficient length for proper PCB mounting. |
| **FR-9** | The capacitor shall have a minimum operational lifetime of 1,000 hours when operating continuously at 40°C ambient temperature with the rated DC working voltage and maximum rated ripple current applied. | **Loss of Function** | • Capacitor fails catastrophically (shorts, opens, vents) before 1,000 hours of operation under the specified life-test conditions. |
| | | **Degraded Function (Under)** | • A primary electrical parameter (e.g., Capacitance, ESR) drifts beyond its end-of-life specification before 1,000 hours. |
| | | **Unintended Function** | • Physical degradation (case bulging, electrolyte leakage) occurs before 1,000 hours. |
| | | **Erroneous Duration (Too short)** | • Fails to operate within any specified parameter for the minimum duration of 1,000 hours. |
| **FR-10** | The capacitor and all its constituent materials shall be compliant with the latest versions of the RoHS and REACH regulations at the time of manufacture. | **Loss of Function** | • Fails to comply with RoHS regulations (contains a restricted substance above threshold).<br>• Fails to comply with REACH regulations (contains a Substance of Very High Concern or lacks required registration). |
| | | **Unintended Function** | • Contains a restricted or hazardous substance not declared in the material declaration datasheet. |
| | | **Erroneous Timing** | • Compliant with an outdated version of RoHS/REACH, but not the "latest version at the time of manufacture." |

In [10]:
print(result.raw)


### Component FMEA: Potential Failure Modes for VFD Capacitor

The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived systematically from the provided functional requirements using the specified guide words.

| Req. ID | Functional Requirement | Generic Failure Mode Category | Specific Failure Mode(s) |
| :--- | :--- | :--- | :--- |
| **FR-1** | The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% at 60 Hz and 35°C. | **Loss of Function** | • Capacitor fails open (capacitance drops to near zero).<br>• Capacitor fails short (dielectric breakdown results in a low-resistance path). |
| | | **Degraded Function (Under)** | • Capacitance is below the lower tolerance limit of 95 µF due to aging, electrolyte evaporation, or internal damage. |
| | | **Degraded Function (Over)** | • Capacitance is above the upper tolerance limit of 105 µF due to manufacturing defect or dielectric property shift. |
| | | **Intermi